In [1]:
import os
import mitsuba as mi

output_data_folder = 'data'
scene_file = 'scenes/cornell-box/scene.xml'

mi.set_variant('scalar_rgb')
scene = mi.load_file(scene_file)

[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [9]:
# clear data if necessary
%rm -r data

In [10]:
# create data folder if necessary
os.makedirs(output_data_folder, exist_ok=True)

In [13]:
params = mi.traverse(scene)
params['logfile'] = 'data/gnn_file_2.path'
params.update();

In [14]:
mi.render(scene, spp=10)

TensorXf(shape=(64, 64, 3))

In [2]:
from mignn.container import SimpleLightGraphContainer
light_graphs1 = SimpleLightGraphContainer.fromfile('data/gnn_file_1.path', scene_file, verbose=True)
light_graphs2 = SimpleLightGraphContainer.fromfile('data/gnn_file_2.path', scene_file, verbose=True)
print(f'{light_graphs1}')
print(f'{light_graphs2}')

Load of `data/gnn_file_1.path` in progress: 100.00%
Load of `data/gnn_file_2.path` in progress: 100.00%
SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 40960, n_nodes: 138231 (duplicate: 0), n_connections: 97271 (built: 0)]
SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 40960, n_nodes: 138231 (duplicate: 0), n_connections: 97271 (built: 0)]


In [3]:
from mignn.manager import LightGraphManager
merged_graph_container = LightGraphManager.fusion([light_graphs1, light_graphs2])
print(merged_graph_container)

SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 81920, n_nodes: 276462 (duplicate: 0), n_connections: 194542 (built: 0)]


In [4]:
merged_graph_container.build_connections(n_graphs=10, n_nodes_per_graphs=2, n_neighbors=5, verbose=True)

/home/jerome/Documents/Research/Development/Rendering/mitsuba3/build-gnn/python/drjit/router.py:1753: RuntimeWarning: divide by zero encountered in divide
  return 1.0 / arg
/home/jerome/Documents/Research/Development/Rendering/mitsuba3/build-gnn/python/drjit/generic.py:194: RuntimeWarning: invalid value encountered in scalar multiply
  ar[i] = a0[i] * a1


In [5]:
print(merged_graph_container)

SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 81920, n_nodes: 313365 (duplicate: 36903), n_connections: 231899 (built: 37357)]


In [6]:
graphs_samples = merged_graph_container.get_graphs((0, 0))
len(graphs_samples)

20

In [7]:
graphs_samples[0].data

{'nodes': [[-0.00167775, 1.00171, 6.79, -0.163149, 0.166625, -0.972429],
  [-0.979138, 2.0, 0.963979, -8.74228e-08, -1.0, -8.74228e-08],
  [-1.0, 1.92422, 0.867575, -1.0, -4.37114e-08, -8.74228e-08],
  [-0.949757, 2.0, 0.817502, -8.74228e-08, -1.0, -8.74228e-08]],
 'edges': [(0, 1, [5.9912]), (1, 2, [0.124219]), (2, 3, [0.10367])],
 'targets': [0.0122338, 0.000872541, 0.000214275]}

In [9]:
import torch
from torch_geometric.data import Data

ModuleNotFoundError: No module named 'torch'